# Exercise II - Telco Wrangle

In [1]:
#import needed libraries

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
from pydataset import data

#import ignore warnings
import warnings
warnings.filterwarnings("ignore")


### Create Acquire Function

In [2]:
#Create helper function to get the necessary connection url.
def get_connection(db_name):
    '''
    This function uses my info from my env file to
    create a connection url to access the Codeup db.
    '''
    from env import host, user, password
    return f'mysql+pymysql://{user}:{password}@{host}/{db_name}'

In [3]:
#Create function to retrieve telco_churn data with specific columns
def get_telco_data():
    '''
    This function reads in the Telco Churn data from the Codeup db
    and returns a pandas DataFrame with customer_id, monthly_charges, tenure, total_charges columns
    for customers who have 2-year contracts.
    '''
    
    sql_query = '''
    SELECT customer_id, monthly_charges, tenure, total_charges
    FROM customers
    WHERE contract_type_id LIKE '3'
    '''
    
    return pd.read_sql(sql_query, get_connection('telco_churn'))

In [4]:
# acquire
from env import host, user, password
import acquire

In [5]:
df = acquire.get_telco_data()

_______________________________

### Prepare Data

In [6]:
df.head()

,customer_id,monthly_charges,tenure,total_charges
0,0013-SMEOE,109.70,71,7904.25
1,0014-BMAQU,84.65,63,5377.8
2,0016-QLJIS,90.45,65,5957.9
3,0017-DINOC,45.20,54,2460.55
4,0017-IUDMW,116.80,72,8456.75


In [7]:
#look at data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1695 entries, 0 to 1694
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      1695 non-null   object 
 1   monthly_charges  1695 non-null   float64
 2   tenure           1695 non-null   int64  
 3   total_charges    1695 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 53.1+ KB


In [8]:
df.shape

(1695, 4)

In [9]:
#descriptive statistics of df
df.describe().T

,count,mean,std,min,25%,50%,75%,max
monthly_charges,1695.0,60.770413,34.678865,18.4,24.025,64.35,90.45,118.75
tenure,1695.0,56.735103,18.209363,0.0,48.000,64.00,71.00,72.00


In [10]:
#check for null values
df.isnull().sum()

#this shows that there are none

customer_id        0
monthly_charges    0
tenure             0
total_charges      0
dtype: int64

In [11]:
#again check for nulls
df.isnull().any()

customer_id        False
monthly_charges    False
tenure             False
total_charges      False
dtype: bool

In [12]:
#replace blank spaces and special characters
df = df.replace(r'^\s*$', np.nan, regex=True)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1695 entries, 0 to 1694
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      1695 non-null   object 
 1   monthly_charges  1695 non-null   float64
 2   tenure           1695 non-null   int64  
 3   total_charges    1685 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 53.1+ KB


In [14]:
#change total_charges to float from object
df.total_charges = df.total_charges.str.replace(' ', '0').astype(float)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1695 entries, 0 to 1694
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      1695 non-null   object 
 1   monthly_charges  1695 non-null   float64
 2   tenure           1695 non-null   int64  
 3   total_charges    1685 non-null   float64
dtypes: float64(2), int64(1), object(1)
memory usage: 53.1+ KB


In [16]:
#find NaN values
df.isna().sum()

#this shows that there are 10 NaN values in total_charges

customer_id         0
monthly_charges     0
tenure              0
total_charges      10
dtype: int64

In [17]:
#fill NaN values (10 out of 1695) with monthly charges
df.total_charges = df.total_charges.fillna(df.monthly_charges)

In [18]:
#double check that dropna worked
df.isna().sum()

#there are now no NaN values

customer_id        0
monthly_charges    0
tenure             0
total_charges      0
dtype: int64

In [19]:
#find the new shape of your dataframe
df.shape

(1695, 4)

________________________________

## Create wrangle function:

In [20]:
def wrangle_telco():
    '''
    This function checks to see if telco_churn.csv already exists, 
    if it does not, one is created
    then the data is cleaned and the dataframe is returned
    '''
    #check to see if telco_churn.csv already exist
    if os.path.isfile('telco_churn.csv'):
        df = pd.read_csv('telco_churn.csv', index_col=0)
    
    else:

        #creates new csv if one does not already exist
        df = get_telco_data()
        df.to_csv('telco_churn.csv')

    #replace blank spaces and special characters
    df = df.replace(r'^\s*$', np.nan, regex=True)

    #change total_charges to float from object
    df.total_charges = df.total_charges.str.replace(' ', '0').astype(float)

    #fill NaN values (10 out of 1695) with monthly charges
    df.total_charges = df.total_charges.fillna(df.monthly_charges)

    return df

In [21]:
df.head()

,customer_id,monthly_charges,tenure,total_charges
0,0013-SMEOE,109.70,71,7904.25
1,0014-BMAQU,84.65,63,5377.80
2,0016-QLJIS,90.45,65,5957.90
3,0017-DINOC,45.20,54,2460.55
4,0017-IUDMW,116.80,72,8456.75


In [22]:
# in future project  just use:
# from wrangle import wrangle_telco
# df=wrangle_telco()

<hr style="border:2px solid black"> </hr>

# Exercise III- Challenge

In [23]:
zillow_df = acquire.get_zillow_data()

In [24]:
zillow_df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0


In [29]:
#get shape of df
zillow_df.shape

(2152863, 7)

In [25]:
#get column names and dtypes
zillow_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2152863 entries, 0 to 2152862
Data columns (total 7 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   bedroomcnt                    float64
 1   bathroomcnt                   float64
 2   calculatedfinishedsquarefeet  float64
 3   taxvaluedollarcnt             float64
 4   yearbuilt                     float64
 5   taxamount                     float64
 6   fips                          float64
dtypes: float64(7)
memory usage: 115.0 MB


In [27]:
#get null count
zillow_df.isnull().sum()

bedroomcnt                        11
bathroomcnt                       11
calculatedfinishedsquarefeet    8484
taxvaluedollarcnt                493
yearbuilt                       9337
taxamount                       4442
fips                               0
dtype: int64

In [30]:
zillow_df.describe()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
count,2.152852e+06,2.152852e+06,2.144379e+06,2.152370e+06,2.143526e+06,2.148421e+06,2.152863e+06
mean,3.287196e+00,2.230688e+00,1.862855e+03,4.618962e+05,1.960950e+03,5.634866e+03,6.048377e+03
std,9.547544e-01,9.992796e-01,1.222125e+03,6.996760e+05,2.216220e+01,8.178910e+03,2.043329e+01
min,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.801000e+03,1.850000e+00,6.037000e+03
25%,3.000000e+00,2.000000e+00,1.257000e+03,1.881702e+05,1.949000e+03,2.534980e+03,6.037000e+03
50%,3.000000e+00,2.000000e+00,1.623000e+03,3.276710e+05,1.958000e+03,4.108950e+03,6.037000e+03
75%,4.000000e+00,3.000000e+00,2.208000e+03,5.345270e+05,1.976000e+03,6.414320e+03,6.059000e+03
max,2.500000e+01,3.200000e+01,9.525760e+05,9.842891e+07,2.016000e+03,1.337756e+06,6.111000e+03


In [31]:
#replace blank spaces and special characters
zillow_df = zillow_df.replace(r'^\s*$', np.nan, regex=True)

In [32]:
#drop Null values
zillow_df = zillow_df.dropna()

In [33]:
#double check that dropna worked
zillow_df.isnull().sum()

bedroomcnt                      0
bathroomcnt                     0
calculatedfinishedsquarefeet    0
taxvaluedollarcnt               0
yearbuilt                       0
taxamount                       0
fips                            0
dtype: int64